In [203]:
import numpy as np
import random
import matplotlib.pyplot as plt
import copy
from pettingzoo import ParallelEnv

num_agents = 10                   
num_rounds = 200
num_episodes = 100                    
b = [2, 5, 10]           # Benefit
c = 1                    # Cost of cooperation

# Learning parameters        
# chi = 10 / num_episodes    # Reputation assignment error
chi = 0.01

num_seeds = 20           # Number of random seeds for each run

In [ ]:
class MatrixGame(ParallelEnv):
    metadata = {'render_modes': ['human']}

    def __init__(self, reward_matrix, agents, norm):
        self.agents = agents 
        self.possible_agents = self.agents[:]
        self.reward_matrix = reward_matrix
        self.norm = norm
        self.last_opponent = {}
        self.actions = {}

    def reset(self):
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0.0 for agent in self.agents}
        self.states = {agent: np.random.choice([0,1]) for agent in self.agents}

        return self.states
    
    def get_action_rules(self, action_rule):
        # Convert rule_id to 4-bits
        bits = [(action_rule >> i) & 1 for i in range(4)]  # bits[0]=LSB, bits[3]=MSB
        return bits
        
    def select_new_action(self, action_rule, focal_state, opponent_state):
        if focal_state == 0 and opponent_state == 0:
            return action_rule[3]  # Bit 3
        elif focal_state == 0 and opponent_state == 1:
            return action_rule[2]  # Bit 2
        elif focal_state == 1 and opponent_state == 0:
            return action_rule[1]  # Bit 1
        else:  # (1,1)
            return action_rule[0]  # Bit 0

    def step(self):
        pairings = []
        players = self.agents.copy()
        for _ in range(num_agents//2):
            index = random.randrange(len(players))
            elem1 = players.pop(index)

            index = random.randrange(len(players))
            elem2 = players.pop(index)

            pairings.append((elem1, elem2))


        for pair in pairings:
            action1 = pair[0].select_action(self.states[pair[1]])
            action2 = pair[1].select_action(self.states[pair[0]])

            if action1==5:
                action1 = self.select_new_action(self.get_action_rules(5), self.states[pair[0]], self.states[pair[1]])

            self.actions[pair[0]] = action1
            self.actions[pair[1]] = action2

            self.last_opponent[pair[0]] = pair[1]
            self.last_opponent[pair[1]] = pair[0]

            # action1, action2 = self.determine_action(pair[0], pair[1], action_rules)

            reward1 = self.reward_matrix[action1][action2]
            reward2 = self.reward_matrix[action2][action1]

            self.rewards[pair[0]] = reward1
            self.rewards[pair[1]] = reward2

            state1 = self.determine_state(action1, self.states[pair[1]])
            state2 = self.determine_state(action2, self.states[pair[0]])

            if (random.random() < chi):
                state1 = 1-state1
            if (random.random() < chi):
                state2 = 1-state2

            self.states[pair[0]] = state1
            self.states[pair[1]] = state2

        return self.states, self.rewards, self.actions

In [205]:
class Qlearner:
    """A Q-learning agent"""

    def __init__(
        self,
        seeded=False,
        action_size=2,
        state_size=2,
        learning_rate=0.01,
        gamma=0.99,
        epsilon=0.1,
    ):
        self.action_size = action_size
        self.state_size = state_size

        self.seeded = seeded
        # initialize the Q-table: (State x Agent Action)
        self.qtable = np.zeros((self.state_size, self.action_size))

        self.learning_rate = learning_rate
        self.gamma = gamma  # discount factor
        self.epsilon = epsilon # exploration

        # tracking rewards/progress:
        self.rewards_this_episode = []  # during an episode, save every time step's reward
        self.episode_total_rewards = []  # each episode, sum the rewards, possibly with a discount factor
        self.average_episode_total_rewards = []  # the average (discounted) episode reward to indicate progress

        self.state_history = []
        self.action_history = []

    def reset_agent(self):
        self.qtable = np.zeros((self.state_size, self.action_size))

    def select_greedy(self, state):
        # np.argmax(self.qtable[state]) will select first entry if two or more Q-values are equal, but we want true randomness:
        return np.random.choice(np.flatnonzero(np.isclose(self.qtable[state], self.qtable[state].max())))

    def select_action(self, state):
        if self.seeded:
            return 5 
        if np.random.rand() < self.epsilon:
            action = random.randrange(self.action_size)
        else:
            action = self.select_greedy(state)
        self.state_history.append(state)
        self.action_history.append(action)
        return action

    def update(self, state, action, new_state, reward, done):
        lr = self.learning_rate
        self.qtable[state, action] += lr * (reward + (not done) * self.gamma * np.max(self.qtable[new_state]) - self.qtable[state, action])

        self.rewards_this_episode.append(reward)

        if done:
            # track total reward:
            episode_reward = self._calculate_episode_reward(self.rewards_this_episode, discount=False)
            self.episode_total_rewards.append(episode_reward)

            k = len(self.average_episode_total_rewards) + 1  # amount of episodes that have passed
            self._calculate_average_episode_reward(k, episode_reward)
            
            # reset the rewards for the next episode:
            self.rewards_this_episode = []

    def _calculate_episode_reward(self, rewards_this_episode, discount=False):
        if discount:
            return sum([self.gamma**i * reward for i, reward in enumerate(rewards_this_episode)])
        return sum(rewards_this_episode)

    def _calculate_average_episode_reward(self, k, episode_reward):
        if k > 1:  # running average is more efficient:
            average_episode_reward = (1 - 1 / k) * self.average_episode_total_rewards[-1] + episode_reward / k
        else:
            average_episode_reward = episode_reward
        self.average_episode_total_rewards.append(average_episode_reward)

    def print_rewards(self, episode, print_epsilon=True, print_q_table=True):
        # print("Episode ", episode + 1)
        print("Total (discounted) reward of this episode: ", self.episode_total_rewards[episode])
        print("Average total reward over all episodes until now: ", self.average_episode_total_rewards[-1])

        print("Epsilon:", self.epsilon) if print_epsilon else None
        print("Q-table: ", self.qtable) if print_q_table else None

In [206]:
def run(payoffs, norm):
    run = 1
    np.random.seed(run)
    random.seed(run)

    agents = [Qlearner(seeded=True) for _ in range(0)] + [Qlearner(seeded=False) for _ in range(10)]
    env = MatrixGame(payoffs, agents, norm)

    for episode in range(num_episodes):
        if episode%100==0:
            print(f"Number of episodes: {episode}")
        obs = env.reset()
        for round in range(num_rounds):
            next_obs, rewards, actions = env.step()
            last_opponent = env.last_opponent

            for agent in agents:
                state = obs[last_opponent[agent]]
                new_state = next_obs[last_opponent[agent]]
                if round == num_rounds-1:
                    agent.update(state, actions[agent], new_state, rewards[agent], done=True)
                    # agent.update(obs[agent], action_rules[agent], next_obs[agent], rewards[agent], done=True)
                    print("Previous state:", obs[agent])
                    print("Reward:", rewards[agent])
                    print("Next state:", next_obs[agent])
                    print()
                else:
                    agent.update(state, actions[agent], new_state, rewards[agent], done=False)
            obs = next_obs

    average_agent_round_payoff = np.zeros(num_episodes//2)
    for agent in agents:
        average_round_payoff = np.array(agent.episode_total_rewards[num_episodes//2:])/num_rounds
        average_agent_round_payoff += average_round_payoff

    b = payoffs[0,1]
    average_agent_round_payoff = (average_agent_round_payoff.sum()/(10*(num_episodes//2)))/(b-c)

    return average_agent_round_payoff

In [ ]:
results = []
colors = ["blue", "red", "blue", "red", "blue", "red",]

effective_norm =[1,0,0,1]
ineffective_norm =[0,0,0,0]

payoffs = np.array([[0, b[0]],[-c, b[0]-c]])
results.append(run(payoffs, effective_norm))
results.append(run(payoffs, ineffective_norm))


payoffs = np.array([[0, b[1]],[-c, b[1]-c]])
results.append(run(payoffs, effective_norm))
results.append(run(payoffs, ineffective_norm))


payoffs = np.array([[0, b[2]],[-c, b[2]-c]])
results.append(run(payoffs, effective_norm))
results.append(run(payoffs, ineffective_norm))

results = np.array(results)
results *= 100

x = np.arange(len(results))

plt.figure()
plt.bar(range(len(results)), results.flatten(), color=colors)
plt.ylim(0,100)
pair_labels = ["2", "5", "10"]
pair_positions = [(x[i] + x[i+1])/2 for i in range(0, len(x), 2)]
plt.xticks(pair_positions, pair_labels)
plt.show()

Number of episodes: 0
Previous state: 0
Reward: 0
Next state: 0

Previous state: 1
Reward: 0
Next state: 1

Previous state: 1
Reward: 1
Next state: 1

Previous state: 0
Reward: 0
Next state: 0

Previous state: 1
Reward: 0
Next state: 1

Previous state: 1
Reward: 0
Next state: 1

Previous state: 0
Reward: 0
Next state: 0

Previous state: 1
Reward: 0
Next state: 1

Previous state: 1
Reward: 1
Next state: 1

Previous state: 0
Reward: 0
Next state: 0

Previous state: 0
Reward: 0
Next state: 0

Previous state: 1
Reward: 0
Next state: 1

Previous state: 0
Reward: 2
Next state: 0

Previous state: 0
Reward: 0
Next state: 0

Previous state: 1
Reward: 0
Next state: 1

Previous state: 1
Reward: 0
Next state: 1

Previous state: 0
Reward: 2
Next state: 0

Previous state: 0
Reward: -1
Next state: 0

Previous state: 0
Reward: 0
Next state: 0

Previous state: 0
Reward: -1
Next state: 0

Previous state: 0
Reward: 0
Next state: 0

Previous state: 1
Reward: 0
Next state: 1

Previous state: 0
Reward: 0
Ne